In [ ]:
import os

In [ ]:
%pwd

In [ ]:
os.chdir("../")

In [ ]:
%pwd

In [129]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    data_path:Path
    train_data_path: Path
    test_data_path: Path
    model_name: Path
    target_column : dict
    

In [130]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories



In [131]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.Logistic_Regression
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            train_data_path=config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            target_column = schema,    
    
            
            
        
        )

        return model_trainer_config

In [132]:
import pandas as pd
import os
from mlProject import logger
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder
from lazypredict.Supervised import LazyClassifier
import joblib


In [138]:
class ModelTrainer:
    def __init__(self, config = ModelTrainerConfig):
        self.config = config
        
    def train_model(self):
        data = pd.read_csv(self.config.data_path)
        train_data = pd.read_csv(self.config.train_data_path, usecols=lambda column: column != 'id')
        test_data = pd.read_csv(self.config.test_data_path, usecols=lambda column: column != 'id')
    
        
        target_column_schema =list(self.config.target_column.keys())[0]
        label_encoder = LabelEncoder()
        data[target_column_schema] = label_encoder.fit_transform(data[target_column_schema])
        train_data[target_column_schema] = label_encoder.transform(train_data[target_column_schema])
        test_data[target_column_schema] = label_encoder.transform(test_data[target_column_schema])

        train_x = train_data.drop(columns=[target_column_schema], axis = 1)
        test_x = test_data.drop(columns=[target_column_schema], axis = 1)
        train_y = train_data[target_column_schema]
        test_y = test_data[target_column_schema]
        
        lr = LogisticRegression()
        lr.fit(train_x, train_y)
        
        joblib.dump(lr,os.path.join(self.config.root_dir, self.config.model_name))
        

        
        
        
        

In [1]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_training = ModelTrainer(config=model_trainer_config)
    model_training.train_model()
    
except Exception as e:
    raise e

NameError: name 'ConfigurationManager' is not defined